<a href="https://colab.research.google.com/github/DashTT/Reto-01-Chagas/blob/main/Reto_1_AndradeVillenasEscobar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/andres-merino/FormatoBaseProyectos/blob/main/Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Carrera de Ciencia de Datos</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático &bull; Reto 1
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2025-01
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>



De acuerdo a la Organización Panamericana de la Salud (OPS), el mal de Chagas es una "afección parasitaria, sistémica, crónica, transmitida por vectores y causada por el protozoario Trypanosoma cruzi, con una firme vinculación con aspectos socioeconómico-culturales deficitarios, considerándosela una enfermedad desatendida.". Los agentes  que propagan esta enfermedad son las chinches de Subfamilia Triatominae, quienes a su vez propagan el parásito Trypanosoma cruzi; el causante de la enfermedad.

En este documento se busca utilizar información sobre las viviendas de Loja del año 2005 para encontrar en estos datos posibles perfiles de viviendas en riesgo de vivir y exponerse a esta enfermedad.

Sabiedo esto, para seleccionar los datos podemos tener varias consideraciones que justamente la Organización Panamericana de Salud da a las personas para evitar el contagio como por ejemplo animales portadores de estas chinches, formas de transmisión y prevensión, entre otros. Con ello se trabajará para la selección, análisis y conclusiones del trabajo a realizar en este documento. Dado que es un jupyter notebook, este solo busca dar el código utilizado en la investigación para su replicación y validación.

---
## <font color='264CC7'> Carga de Librerías </font>

Los paquetes necesarios son:

In [ ]:
# Paquetes necesarios
import pandas as pd

---
## <font color='264CC7'> Lectura de datos </font>


Primero, empezamos con la carga de los datos.

In [ ]:
# Cargar datos
df = pd.read_csv("https://github.com/DashTT/Reto-01-Chagas/blob/main/Conjunto%20de%20datos%20Viviendas_Loja_2005-20251215/Viviendas_Loja_2005.csv?raw=true")
display(df.head())

,Código,Status entomologico,Estado,Localidad,Altitud,Longitud,Latitud,A2. Cuantos dormitorios tiene la vivienda,A3 Número de personas que duermen en la vivienda,A4. Duermen 3 o mas personas en algun dormitorio,...,J3 Cuántas,J4. Palma Africana,J4 Distancia de la casa (metros),J4 Cuántas,J5. Otro tipo de de palmera o palma,J5 Cuál,J5 Distancia de la casa (metros),J5 Cuántas,E10 Cuántos,Unnamed: 138
0,AB007,Negativa,Examinada,Algarrobillo,0.0,NaN,NaN,1.0,5.0,-1.0,...,0.0,2.0,0.0,0.0,2.0,NaN,0.0,0.0,2.0,NaN
1,AB207,Negativa,Examinada,Algarrobillo,0.0,NaN,NaN,1.0,1.0,-1.0,...,0.0,2.0,0.0,0.0,2.0,NaN,0.0,0.0,0.0,NaN
2,AB208,Negativa,Examinada,Algarrobillo,0.0,NaN,NaN,1.0,5.0,-1.0,...,0.0,2.0,0.0,0.0,2.0,NaN,0.0,0.0,0.0,NaN
3,AB209,Negativa,Examinada,Algarrobillo,0.0,NaN,NaN,1.0,3.0,-1.0,...,0.0,2.0,0.0,0.0,2.0,NaN,0.0,0.0,0.0,NaN
4,AB210,Negativa,Examinada,Algarrobillo,0.0,NaN,NaN,2.0,10.0,-1.0,...,0.0,2.0,0.0,0.0,2.0,NaN,0.0,0.0,0.0,NaN
